In [1]:
from transformers import BertTokenizer, BertForSequenceClassification

In [2]:
pretrained_model_name='bert-base-chinese'
# 无良好网络环境请提前下载好预训练模型
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name)
model = BertForSequenceClassification.from_pretrained(pretrained_model_name, num_labels=3)

Downloading:   0%|          | 0.00/107k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/624 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/393M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [3]:
import pandas
from tools import *

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 2.526 seconds.
Prefix dict has been built successfully.
W1228 16:32:31.530217 155781 init.cc:157] AVX is available, Please re-compile on local machine
Paddle enabled successfully......


In [4]:
rawdata_path='./data/for_modeling.xlsx'
with open(rawdata_path, 'r',encoding='ISO-8859-1') as fp:
    sheets=pandas.read_excel(rawdata_path,sheet_name=None,index_col=0)

sheet1=sheets['Sheet1'] # 表格一，好评 2017*14
sheet2=sheets['Sheet2'] # 表格二，差评 2084*14
sheet=sheet1.append(sheet2) # 合并好评差评 4101*14
appearance, function, service = split_dimentions(
    sheet
)

In [5]:
appearance_sentences = appearance.values[:, 0]
function_sentences = function.values[:, 0]
service_sentences = service.values[:, 0]


In [63]:
from sklearn.model_selection import train_test_split
import numpy as np

In [66]:
def array2dataset(data_array, lable: int, test_size: float = 0.1):
    train, test = train_test_split(data_array, test_size=test_size)
    return np.stack((train, np.ones(len(train), dtype=np.int8) * lable)), np.stack(
        (test, np.ones(len(test), dtype=np.int8) * lable)
    )


appearance_train, appearance_test = array2dataset(appearance_sentences, 0)
function_train, function_test = array2dataset(function_sentences, 1)
service_train, service_test = array2dataset(service_sentences, 2)


In [67]:
train_set = np.concatenate(
    (
        appearance_train,
        function_train,
        service_train,
    ),
    axis=1,
)
test_set = np.concatenate(
    (
        appearance_test,
        function_test,
        service_test,
    ),
    axis=1,
)


In [75]:
DataFrame(train_set.transpose()).to_csv('data/bert/train.csv',sep='\t',index=0,header=0)
DataFrame(test_set.transpose()).to_csv('data/bert/test.csv',sep='\t',index=0,header=0)

In [30]:
import torch
torch.LongTensor([[1,0,0]]).size()

torch.Size([1, 3])

In [39]:
output = model(**tokenizer('特别特别的不好	颜色也不好看', return_tensors="pt"),labels=torch.Tensor([[1,0,0]]) )


In [42]:
output

SequenceClassifierOutput(loss=tensor(0.7447, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>), logits=tensor([[-0.0488,  0.0535,  0.1962]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [80]:
import torch
from sklearn import metrics
true=torch.tensor([1,0,1,1,1,0,0])
pred=torch.tensor([1,0,1,0,1,0,0])
metrics.f1_score(true,pred),\
metrics.f1_score(true,pred, average='micro'),\
metrics.precision_score(true,pred),\
metrics.recall_score(true,pred),\
metrics.accuracy_score(true,pred)

(0.8571428571428571, 0.8571428571428571, 1.0, 0.75, 0.8571428571428571)

In [15]:
m=map(tokenizer.tokenize,appearance_sentences)

[(_,tokenizer.convert_tokens_to_ids(_)) for _ in m]

[(['颜', '值', '很', '高'], [7582, 966, 2523, 7770]),
 (['鞋', '样', '子', '很', '好', '看'], [7490, 3416, 2094, 2523, 1962, 4692]),
 (['颜', '色', '很', '正', '，', '涂', '上', '很', '显', '女', '王', '气', '质'],
  [7582,
   5682,
   2523,
   3633,
   8024,
   3864,
   677,
   2523,
   3227,
   1957,
   4374,
   3698,
   6574]),
 (['颜', '色', '很', '正', '，', '薄', '涂', '很', '显', '色', '度', '，'],
  [7582, 5682, 2523, 3633, 8024, 5946, 3864, 2523, 3227, 5682, 2428, 8024]),
 (['感',
   '觉',
   '哑',
   '光',
   '款',
   '更',
   '适',
   '合',
   '自',
   '己',
   '，',
   '更',
   '加',
   '时',
   '尚',
   '更',
   '打',
   '底',
   '具',
   '气',
   '场',
   '，',
   '颜',
   '色',
   '很',
   '正',
   '的',
   '大',
   '红',
   '色',
   '，',
   '很',
   '显',
   '气',
   '质'],
  [2697,
   6230,
   1514,
   1045,
   3621,
   3291,
   6844,
   1394,
   5632,
   2346,
   8024,
   3291,
   1217,
   3198,
   2213,
   3291,
   2802,
   2419,
   1072,
   3698,
   1767,
   8024,
   7582,
   5682,
   2523,
   3633,
   4638,
   1920,
   5273,
   5682

In [17]:
tokenizer.convert_tokens_to_ids(['[CLS]', '产', '品', '黑', '色', '高', '端', '大', '气', '上', '档', '次','[SEP]'])

[101, 772, 1501, 7946, 5682, 7770, 4999, 1920, 3698, 677, 3440, 3613, 102]